# Retrograde Analyse für das Schach-Endspiel
_Beschreibung_

Bekannte Probleme:
* Nicht optimiert für ein Spielbrett mit Bauern

Mögliche Verbesserungen:
* Das Initialisieren des Notebooks in Funktionen kapseln, um erneute Ausführungen einfacher zu gestalten.

## Imports
Für das Notebook benötigte Imports
* chess: Die Python-Schach-Bibliothek, mit welcher Schachbretter dargestellt werden, Züge und Zustände ausgewertet etc.
* Factorial aus der Math-Library: Die Fakultät (n!) wird zum Aufstellen der Board_List benötigt
* Clear_output aus der IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen 
verwendet wird.

In [ ]:
import chess
from math import factorial
from IPython.display import clear_output
import json


## Eingabe des Schachbretts, mit der Ausgangssituation
Langfristig wird dies durch [@NicoMiller's](https://github.com/millerni) Board-Setup-Code ersetzt.
Aktuell wird eine Fen eingegeben.
Mit der Forsyth-Edwards-Notation kann der Zustand eines Schachbretts kompakt notiert werden. Mehr Infos zum Thema Fen 
finden sich [hier](https://www.chess.com/terms/fen-chess).

Es stehen folgende Test-Cases zur Verfügung:
* 1: Turm und König gegen König (``8/8/8/8/4k3/8/8/K6R w - - 0 1``)
* 2: 2 Läufer und König gegen König (``4k3/8/8/8/8/8/8/2B1KB2 w - - 0 1``)

Weiter wird in der folgenden Zelle das erste Board-Objekt erstellt.


In [ ]:
# Test Case 1
#fen = "8/8/8/8/4k3/8/8/K6R w - - 0 1"

# Test Case 2
fen = "4k3/8/8/8/8/8/8/2B1KB2 w - - 0 1"

# Test Case with Pawn
#fen = "8/8/8/8/4k3/8/2P5/K7 w - - 0 1"
board = chess.Board(fen)

## Liste der Positionen auf einem Brett (_Square-Names_) bestimmen

In [ ]:
all_positions = []

column_convert = {
    1 : 'a',
    2 : 'b',
    3 : 'c',
    4 : 'd',
    5 : 'e',
    6 : 'f',
    7 : 'g',
    8 : 'h'
}

all_squares = []
for row in range(1,9):
    for col_num in range(1,9):
        column = column_convert[col_num]
        all_squares.append(column + str(row))
print(all_squares)

## Einstellen der Figuren, welche Platziert werden sollen

In [ ]:
# 1 = White, 0 = Black
figures_to_place = [(chess.ROOK, 1), (chess.KING, 1), (chess.KING, 0)]

## Die *place\_figure\_everywhere\_on\_every\_board* Hilfsfunktion

In [ ]:
def place_figure_everywhere_on_every_board(figure, list_of_boards):
    result_list = []
    for board in list_of_boards:
        cur_piece_map = board.piece_map()
        squares_used = list(cur_piece_map.keys())
        for square in all_squares:
            parsed_square = chess.parse_square(square)
            if parsed_square not in squares_used:
                tmp_board = board.copy()
                tmp_board.set_piece_at(parsed_square, figure)
                result_list.append(tmp_board)
    return result_list

## Die *check_boards* Hilfsfunktion

In [ ]:
def check_boards(board_list):
    result_list = []
    for board in board_list:
        if board.is_valid():
            result_list.append(board)
    return result_list

## Create S

In [ ]:
empty_board = chess.Board().empty()
white_boards = [empty_board]
for fig in figures_to_place:
    white_boards = place_figure_everywhere_on_every_board(chess.Piece(fig[0], fig[1]), white_boards)
    print(len(white_boards))

# All checked boards board.turn = White
S_White = check_boards(white_boards)
S_White_Ascii = [x.__str__() for x in S_White]
print(len(S_White))

empty_board.turn = chess.BLACK
black_boards = [empty_board]
for fig in figures_to_place:
    black_boards = place_figure_everywhere_on_every_board(chess.Piece(fig[0], fig[1]), black_boards)
    print(len(black_boards))

# All checked boards board.turn = White
S_Black = check_boards(black_boards)
S_Black_Ascii = [x.__str__() for x in S_Black]
print(len(S_Black))

S = S_White + S_Black
print(len(S))
S_Ascii = S_White_Ascii + S_Black_Ascii
print(len(S_Ascii))

## Gewonnene Spielbretter finden und zu S\_0 hinzufügen
* Über den Inhalt von S iterieren.
* Status-Informationen von der Bibliothek abfragen und überprüfen, ob ein Spieler gewonnen hat.
* Wenn ja, das Objekt zu *S\_0* hinzufügen.
  Weiter werden die Objekte in ihrer String-Representation zu *S\_0\_ASCII* hinzugefügt.
* Wenn nicht, das Objekt in *S\_tmp* hinzufügen. (Dies ist effizienter als das Objekt aus der Liste S zu entfernen)
* Die ursprüngliche Liste S mit *S\_tmp* ersetzen.

In [ ]:
S_tmp = []
S_ASCII = []
S_0 = []
S_0_ASCII = []
S_WHITE_ASCII = []

for board in S:
    outcome = board.outcome()
    # If True Game has finished in a certain way
    if outcome is not None:
        # A winner has been determined
        if outcome.winner is not None and board.is_valid():
            S_0.append(board)
            S_0_ASCII.append((board.turn,board.__str__()))
    # Every other board
    else:
        S_tmp.append(board)
        S_ASCII.append((board.turn, board.__str__()))
S = S_tmp



## OneStepAway Funktion
Diese Funktion wählt (aus zwei Listen mit Board-Objekten) die Spielbretter aus der zweiten Liste, welche einen 
Spielzug von einem Spielbrett in der ersten Liste entfernt sind.

Funktionsargumente:
* S_n: Die Liste mit Board-Objekten, deren Spielbretter sich n Züge vom Sieg entfernt befinden
* S: Die Liste, aus welcher die nächste Stufe ausgewählt werden soll

Nebeneffekte:
Keine.
Alle als Parameter übergebenen Listen bleiben unverändert.

Algorithmus:
* Für effizientere Vergleiche für jedes Objekt in *S* die ASCII-Representation speichern.
* Über die Objekte in *S_n* iterieren
  * Den Spieler, welcher am Zug ist wechseln (Da um im aktuellen Zustand anzukommen der andere Spieler einen Zug 
  gemacht hat)
  * pseudo legale Spielzüge von der Bibliothek berechnen Lassen
    Ein pseudo legaler Spielzug ist ein Spielzug, welcher die grundsätzlichen Bewegungsregeln der Figur einhält,
    aber das Schachbrett unter Umständen in einem nicht regelkonformen Zustand versetzt.
  * Über diese Spielzüge iterieren
    * Den Zug ausführen
    * Überprüfen, ob das Board sich in einem erlaubten Zustand befindet.
      Wenn nein, den Schleifendurchlauf abbrechen.
    * Mittels der ASCII-Representation überprüfen, ob das modifizierte Spielbrett in *S* gefunden wird.
    * Wenn es gefunden wird, die ASCII-Representation zu *s_n1_ascii* hinzufügen
    * Den Zug rückgängig machen
  * Den Spieler, welcher ursprünglich am Zug war wiederherstellen.
* Über *S* iterieren
  * Wenn sich das Spielbrett in *s_n1_ascii* befindet, wird es *s_n1* hinzugefügt.
  * Wenn sich das Spielbrett nicht in *s_n1_ascii* befindet, wird es *s_tmp* hinzugefügt.
* *s_n1* und *s_tmp* als Tupel zurückgeben.

In [31]:
def one_step_away(S_n, S, s_white_ascii, iterationCount):
    #variables
    s_n1 = []
    s_n1_tmp = []
    s_n1_ascii = []
    s_ascii = []
    s_n_ascii = []
    s_tmp = []

    #create temporary list for comparison
    for chessBoard in S:
        s_ascii.append((chessBoard.turn, chessBoard.__str__()))

    for chessBoard in S_n:
        s_n_ascii.append((chessBoard.turn, chessBoard.__str__()))

    for i in range(len(S_n)):

        if iterationCount is not None:
            status = "Calculating S" + str(iterationCount) + " - Board " + str(i+1) + " of " + str(len(S_n)) + \
                     " from S" + str(iterationCount-1)
        else:
            status = "Board " + str(i+1) + " of " + str(len(S_n))
        clear_output(wait=True)
        print(status)

        # Get current board and invert the player
        chessBoard = S_n[i]
        chessBoard.turn = chessBoard.turn ^ True

        # Find all Pawns
        pawn_at = []
        for pawn in chessBoard.pieces(chess.PAWN, True):
            pawn_at.append(chess.square_name(pawn))

        # try moves and check if they remain in S
        for pLMove in chessBoard.pseudo_legal_moves:
            if chess.square_name(pLMove.from_square) not in pawn_at:
                chessBoard.push(pLMove)
                if not chessBoard.is_valid():
                    chessBoard.pop()
                    continue
                # If the new board is found in S, it can be reached in one step
                if (chessBoard.turn ^ True, chessBoard.__str__()) in s_ascii:
                    s_n1_ascii.append((chessBoard.turn ^ True, chessBoard.__str__()))
                chessBoard.pop()
        '''
        * As pawns only move forward their move cannot cannot be reversed through the pseudo legal moves
        * to mitigate this the pawn is moved back one row manually
        '''
        for pawn in chessBoard.pieces(chess.PAWN, True):
            if chessBoard.piece_at(pawn - 9) is None:
                chessBoard.remove_piece_at(pawn)
                chessBoard.set_piece_at(pawn - 9, chess.Piece.from_symbol('P'))
                print(chessBoard)
                if chessBoard.is_valid():
                    if (chessBoard.turn ^ True, chessBoard.__str__()) in s_ascii:
                        s_n1_ascii.append((chessBoard.turn ^ True, chessBoard.__str__()))
                chessBoard.remove_piece_at(pawn - 9)
                chessBoard.set_piece_at(pawn, chess.Piece.from_symbol('P'))
        '''
        * As pawns can be exchanged to other figures and this cannot be undone this exchange can also not be reversed
        * through the pseudo legal moves.
        * To mitigate this a pawn will be placed in the top row in the spot of a queen
        '''
        queen = check_top_row(chessBoard)
        if queen:
            chessBoard.remove_piece_at(queen)
            chessBoard.set_piece_at(queen, chess.Piece.from_symbol('P'))
            if chessBoard.is_valid():
                if (chessBoard.turn ^ True, chessBoard.__str__()) in s_ascii:
                    s_n1_ascii.append((chessBoard.turn ^ True, chessBoard.__str__()))
            chessBoard.remove_piece_at(queen)
            chessBoard.set_piece_at(queen, chess.Piece.from_symbol('Q'))

        # Restore the original state of the board
        chessBoard.turn = chessBoard.turn ^ True

    clear_output(wait=True)
    if iterationCount is not None:
        print("Calculating S" + str(iterationCount) + " - Mapping ASCII boards to board objects")
    else:
        print("Mapping ASCII boards to board objects")
    #create return lists
    for chessBoard in S:
        if(chessBoard.turn, chessBoard.__str__()) in s_n1_ascii:
            s_n1.append(chessBoard)
        else:
            s_tmp.append(chessBoard)


    clear_output(wait=True)
    if iterationCount is not None:
        print("Calculating S" + str(iterationCount) + " - Checking Black Moves for determinism")
    else:
        print("Checking Black Moves for determinism")

    '''
    * We can choose every turn white does, but not for black.
    * If one particular move from black brings the board from S_n to S_n-1 it is not guaranteed that every move
    * black can do does so as well.
    * To mitigate this only boards where every move ends up in a lower denomination of S will be added to the currently
    * calculated one.
    * For efficiency an ascii representation is carried through all calculations.
    '''
    # Only needed for Black-Moves
    for chessBoard in s_n1:
        include = True
        if not chessBoard.turn:
            for move in chessBoard.legal_moves:
                chessBoard.push(move)
                str_rep = (chessBoard.turn, chessBoard.__str__())
                chessBoard.pop()
                if str_rep not in s_white_ascii:
                    include = False
        if include:
            s_n1_tmp.append(chessBoard)
        else:
            s_tmp.append(chessBoard)
    s_n1 = s_n1_tmp

    # Only needed for White-Moves
    if iterationCount % 2 == 1:
        s_white_ascii = s_white_ascii + s_n1_ascii


    clear_output(wait=True)
    print("Done with S" + str(iterationCount) + " - " + str(len(s_tmp)) + " Boards remaining in S")



    return s_n1, s_tmp, s_white_ascii

## Die *check\_top\_row* Hilfsfunktion
Diese Funktion überprüft, ob sich in der obersten Reihe eines Boards eine Königin befindet und gibt ggf. 
die Quadrat-Nummer zurück.

In [ ]:
def check_top_row(board):
    for i in range(56, 64):
        if board.piece_type_at(i) == chess.QUEEN:
            return i

        return False

## In einer Schleife die Liste mit S_n berechnen
Solange Boards einem n zugeordnet werden, wird die Schleife weitergeführt. Sie wird abgebrochen, wenn S_n+1 leer ist.

In [ ]:
S_n_sequence = [S_0]
while True:

    S_n_new, S, S_WHITE_ASCII = one_step_away(S_n_sequence[-1], S, S_WHITE_ASCII, len(S_n_sequence))
    if not S_n_new: #an empty list is false
        break
    S_n_sequence.append(S_n_new)

print("Abgeschlossen")
print(str(len(S_n_sequence)) + " S gefunden")
print(str(len(S)) + " Boards konnten nicht zugeordnet werden.")

## Ergebnis in Datei schreiben

In [ ]:
S_n_seq_ascii = []
S_ascii = []

for i in range(len(S_n_sequence)):
    S_n_ascii = []
    for board in S_n_sequence[i]:
        S_n_ascii.append(board.fen())
    S_n_seq_ascii.append(S_n_ascii)

for board in S:
    S_ascii.append(board.fen())

f = open("S_n_seq.json", "w")
f.write(json.dumps(S_n_seq_ascii))
f.close()

f = open("S_rem.json", "w")
f.write(json.dumps(S_ascii))
f.close()